Given the `metadata/countries.json` file created by `Prepare Sources`, fetch each of the referenced linked data resources.

We'll just RDFlib to parse the RDF and serialize it back out as one big file.  We could just use `Graph.parse()` to directly fetch and parse the source data, but to cache the response, we'll fetch the RDF and put it in the `in` folder if it's not there already.

In [ ]:
import json
import requests
from pathlib import Path
from rdflib import Graph

countries_json = Path('metadata') / 'countries.json'

countries = json.load(open('metadata/countries.json'))
countriesGraph = Graph()

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

for country, tgn_list in list(countries.items()):
    if len(tgn_list) > 0:
        tgn_id = tgn_list[0][len('http://vocab.getty.edu/tgn/'):]
        sourceFile = sourceFolder / (tgn_id + '.ttl')
        if not(sourceFile.exists() and sourceFile.is_file()):
            response = requests.get(tgn_list[0], headers={'Accept': 'text/turtle'})
            with open(sourceFile, 'wb') as f:
                f.write(response.content)
        countriesGraph.parse(open(sourceFile), format='turtle')

destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True)

countries_ttl = destinationFolder / 'countries.ttl'
with open(countries_ttl, 'wb') as o:
    countriesGraph.serialize(destination=o, format='turtle')